# Extract introns and genome coordinates of genes from GFF file fata
The purpose of this script is to extract intron coordinates and gene coordinates from the GFF file data
We will look at features labeled exons and grab the end coordinates and start coordinates.

In [87]:
import pandas as pd

In [88]:
data_genes = pd.read_csv("../data/GRCh38_latest_genomic_ValidChroms_OnlyGenes.bed",sep="\t",header=None)
print data_genes.shape
data_genes.head()

(2017619, 8)


,0,1,2,3,4,5,6,7
0,chr1,11874,12227,+,DDX11L1,NR_046018.2,BestRefSeq,exon
1,chr1,11874,14409,+,DDX11L1,NR_046018.2,BestRefSeq,transcript
2,chr1,12613,12721,+,DDX11L1,NR_046018.2,BestRefSeq,exon
3,chr1,13221,14409,+,DDX11L1,NR_046018.2,BestRefSeq,exon
4,chr1,14362,14829,-,WASH7P,NR_024540.1,BestRefSeq,exon


In [90]:
# Get number of unique genes and transcript IDs 
genes_transcripts = data_genes.iloc[:,[4,5]]
genes_transcripts_uniq = genes_transcripts.drop_duplicates()
print genes_transcripts_uniq.shape
genes_transcripts_uniq.head()

(157829, 2)


,4,5
0,DDX11L1,NR_046018.2
4,WASH7P,NR_024540.1
11,MIR6859-1,NR_106918.1
18,MIR1302-2HG,XR_001737835.1
20,MIR1302-2,NR_036051.1


In [91]:
# Set column names
data_genes.columns = ["chrm","start","end","strand","gene","transcriptID","RefSeqType","feature"]

In [92]:
# Only grab exons 
data_exons = data_genes[data_genes["feature"]=="exon"]
print data_exons.shape
data_exons.head()

(1859728, 8)


,chrm,start,end,strand,gene,transcriptID,RefSeqType,feature
0,chr1,11874,12227,+,DDX11L1,NR_046018.2,BestRefSeq,exon
2,chr1,12613,12721,+,DDX11L1,NR_046018.2,BestRefSeq,exon
3,chr1,13221,14409,+,DDX11L1,NR_046018.2,BestRefSeq,exon
4,chr1,14362,14829,-,WASH7P,NR_024540.1,BestRefSeq,exon
6,chr1,14970,15038,-,WASH7P,NR_024540.1,BestRefSeq,exon


In [93]:
data_exons.iloc[:,[0,1,2,5,3]].to_csv("../data/ExtractedExonCoordinatesFromGFFfile.bed",sep="\t",header=False,index=False)

## Let's get the exon-intron boundary coordinates from this exon data

In [19]:
# Split up exons by plus and minus strand
data_exons_plus = data_exons[data_exons["strand"]=="+"]
print data_exons_plus.head()
data_exons_minus = data_exons[data_exons["strand"]=="-"]
print data_exons_minus.head()

    chrm  start    end strand         gene    transcriptID  RefSeqType feature
0   chr1  11874  12227      +      DDX11L1     NR_046018.2  BestRefSeq    exon
2   chr1  12613  12721      +      DDX11L1     NR_046018.2  BestRefSeq    exon
3   chr1  13221  14409      +      DDX11L1     NR_046018.2  BestRefSeq    exon
18  chr1  29926  30039      +  MIR1302-2HG  XR_001737835.1      Gnomon    exon
21  chr1  30366  30503      +    MIR1302-2     NR_036051.1  BestRefSeq    exon
   chrm  start    end strand    gene transcriptID  RefSeqType feature
4  chr1  14362  14829      -  WASH7P  NR_024540.1  BestRefSeq    exon
6  chr1  14970  15038      -  WASH7P  NR_024540.1  BestRefSeq    exon
7  chr1  15796  15947      -  WASH7P  NR_024540.1  BestRefSeq    exon
8  chr1  16607  16765      -  WASH7P  NR_024540.1  BestRefSeq    exon
9  chr1  16858  17055      -  WASH7P  NR_024540.1  BestRefSeq    exon


In [20]:
data_exons_plus = data_exons_plus.reset_index(drop=True)
data_exons_minus = data_exons_minus.reset_index(drop=True)

In [21]:
# Get 5' exon-intron boundaries
data_5p = pd.DataFrame({"chrm":pd.concat([data_exons_plus["chrm"],data_exons_minus["chrm"]]),"start":pd.concat([data_exons_plus["end"]-100,data_exons_minus["start"]-100]),
                       "end":pd.concat([data_exons_plus["end"]+100,data_exons_minus["start"]+100]), "strand":pd.concat([data_exons_plus["strand"],data_exons_minus["strand"]]),
                       "transcriptID":pd.concat([data_exons_plus["transcriptID"],data_exons_minus["transcriptID"]]), "gene":pd.concat([data_exons_plus["gene"],data_exons_minus["gene"]])},
                       columns=["chrm","start","end","transcriptID","strand","gene"])
print data_5p.head()
print data_5p.tail()

   chrm  start    end    transcriptID strand         gene
0  chr1  12127  12327     NR_046018.2      +      DDX11L1
1  chr1  12621  12821     NR_046018.2      +      DDX11L1
2  chr1  14309  14509     NR_046018.2      +      DDX11L1
3  chr1  29939  30139  XR_001737835.1      +  MIR1302-2HG
4  chr1  30403  30603     NR_036051.1      +    MIR1302-2
        chrm      start        end    transcriptID strand          gene
919719  chr9  138044779  138044979     NR_121582.1      -  LOC101928786
919720  chr9  138199833  138200033     NR_135133.1      -  LOC101928932
919721  chr9  138203038  138203238     NR_135133.1      -  LOC101928932
919722  chr9  138215881  138216081  XR_001746987.2      -  LOC107987145
919723  chr9  138216445  138216645  XR_001746987.2      -  LOC107987145


In [30]:
# Get 3' exon-intron boundaries
data_3p = pd.DataFrame({"chrm":pd.concat([data_exons_plus["chrm"],data_exons_minus["chrm"]]),"start":pd.concat([data_exons_plus["start"]-100,data_exons_minus["end"]-100]),
                       "end":pd.concat([data_exons_plus["start"]+100,data_exons_minus["end"]+100]), "strand":pd.concat([data_exons_plus["strand"],data_exons_minus["strand"]]),
                       "transcriptID":pd.concat([data_exons_plus["transcriptID"],data_exons_minus["transcriptID"]]), "gene":pd.concat([data_exons_plus["gene"],data_exons_minus["gene"]])},
                       columns=["chrm","start","end","transcriptID","strand","gene"])
print data_3p.head()
print data_3p.tail()

   chrm  start    end    transcriptID strand         gene
0  chr1  11774  11974     NR_046018.2      +      DDX11L1
1  chr1  12513  12713     NR_046018.2      +      DDX11L1
2  chr1  13121  13321     NR_046018.2      +      DDX11L1
3  chr1  29826  30026  XR_001737835.1      +  MIR1302-2HG
4  chr1  30266  30466     NR_036051.1      +    MIR1302-2
        chrm      start        end    transcriptID strand          gene
919719  chr9  138044971  138045171     NR_121582.1      -  LOC101928786
919720  chr9  138200182  138200382     NR_135133.1      -  LOC101928932
919721  chr9  138203353  138203553     NR_135133.1      -  LOC101928932
919722  chr9  138216329  138216529  XR_001746987.2      -  LOC107987145
919723  chr9  138216553  138216753  XR_001746987.2      -  LOC107987145


In [37]:
# 5p exon-intron boundaries grouped
data_5p_groupedByTranscript = data_5p.groupby(by=["transcriptID"])
# 3p exon-intron boundaries grouped
data_3p_groupedByTranscript = data_3p.groupby(by=["transcriptID"])
# All exons grouped
data_exons_groupedByTranscript = data_exons.groupby(by=["transcriptID"])

all_5p_ExonIntronCoords_ToWrite = pd.DataFrame(columns=["chrm","start","end","transcriptID","strand"])
all_3p_ExonIntronCoords_ToWrite = pd.DataFrame(columns=["chrm","start","end","transcriptID","strand"])
new_geneNameDF = pd.DataFrame(columns=["TranscriptID","Gene"])

for key, item in data_exons_groupedByTranscript:
    
    coords_key = data_exons_groupedByTranscript.get_group(key)
    num_rows = coords_key.shape[0]
    transcriptIDs = [key+"_"+str(i+1) for i in range(num_rows-1)]
    
    exonintron_coords_5p = data_5p_groupedByTranscript.get_group(key)
    exonintron_coords_5p_sorted = exonintron_coords_5p.sort_values(by=["chrm","start"])
    # We are getting rid of the first row for 5p, because that is not actually including the intron, it's the genomic region
    #print list(exonintron_coords_5p_sorted["strand"].values)
    if list(exonintron_coords_5p_sorted["strand"].values)[0]=="+":
        exonintron_coords_5p_sorted = exonintron_coords_5p_sorted.iloc[:-1]
    else:
        exonintron_coords_5p_sorted = exonintron_coords_5p_sorted.iloc[1:]
    new_DF_5p = pd.DataFrame({"chrm":exonintron_coords_5p_sorted["chrm"],"start":exonintron_coords_5p_sorted["start"],"end":exonintron_coords_5p_sorted["end"],"transcriptID":transcriptIDs,"strand":exonintron_coords_5p_sorted["strand"]}
                          ,columns=["chrm","start","end","transcriptID","strand"])
    all_5p_ExonIntronCoords_ToWrite = pd.concat([all_5p_ExonIntronCoords_ToWrite,new_DF_5p],ignore_index=True,axis=0)
    all_5p_ExonIntronCoords_ToWrite.reset_index(drop=True)
    
    exonintron_coords_3p = data_3p_groupedByTranscript.get_group(key)
    exonintron_coords_3p_sorted = exonintron_coords_3p.sort_values(by=["chrm","start"])
    # We are getting rid of the last row for 3p, because that is not actually including the intron, it's the genomic region
    if list(exonintron_coords_3p_sorted["strand"].values)[0]=="+":
        exonintron_coords_3p_sorted = exonintron_coords_3p_sorted.iloc[1:]
    else:
        exonintron_coords_3p_sorted = exonintron_coords_3p_sorted.iloc[:-1]
    new_DF_3p = pd.DataFrame({"chrm":exonintron_coords_3p_sorted["chrm"],"start":exonintron_coords_3p_sorted["start"],"end":exonintron_coords_3p_sorted["end"],"transcriptID":transcriptIDs,"strand":exonintron_coords_3p_sorted["strand"]}
                          ,columns=["chrm","start","end","transcriptID","strand"])
    all_3p_ExonIntronCoords_ToWrite = pd.concat([all_3p_ExonIntronCoords_ToWrite,new_DF_3p],ignore_index=True,axis=0)
    all_3p_ExonIntronCoords_ToWrite.reset_index(drop=True)
    
    new_geneNameDF = pd.concat([new_geneNameDF,pd.DataFrame({"Gene":list(coords_key["gene"].values)[1:],"TranscriptID":transcriptIDs},columns=["TranscriptID","Gene"])],ignore_index=True,axis=0)
    new_geneNameDF.reset_index(drop=True)
    
new_geneNameDF.to_csv("../data/ExonIntronCoords_GeneNameTranscriptID_NCBI_RefSeq_hg38_FromGFFfile.tsv",sep="\t",header=False,index=False)

all_5p_ExonIntronCoords_ToWrite["chrm"] = all_5p_ExonIntronCoords_ToWrite["chrm"].replace("chr23","chrX")
all_5p_ExonIntronCoords_ToWrite["chrm"] = all_5p_ExonIntronCoords_ToWrite["chrm"].replace("chr24","chrY")


all_5p_ExonIntronCoords_ToWrite_Sorted = all_5p_ExonIntronCoords_ToWrite.sort_values(by=["chrm","start"])
all_5p_ExonIntronCoords_ToWrite_Sorted.to_csv("../data/ExtractedExonIntronCoordinates_5p_FromGFFfile_ForSequence.bed",sep="\t",header=False,index=False)

all_3p_ExonIntronCoords_ToWrite["chrm"] = all_3p_ExonIntronCoords_ToWrite["chrm"].replace("chr23","chrX")
all_3p_ExonIntronCoords_ToWrite["chrm"] = all_3p_ExonIntronCoords_ToWrite["chrm"].replace("chr24","chrY")


all_3p_ExonIntronCoords_ToWrite_Sorted = all_3p_ExonIntronCoords_ToWrite.sort_values(by=["chrm","start"])
all_3p_ExonIntronCoords_ToWrite_Sorted.to_csv("../data/ExtractedExonIntronCoordinates_3p_FromGFFfile_ForSequence.bed",sep="\t",header=False,index=False)

KeyboardInterrupt: 

In [39]:
all_5p_ExonIntronCoords_ToWrite.sort_values(by=["chrm","start"]).head()

,chrm,start,end,transcriptID,strand
63426,chr1,999426,999626,NM_001142467.1_1,-
63427,chr1,999592,999792,NM_001142467.1_2,-
53638,chr1,1173826,1174026,NM_001130045.1_1,+
53639,chr1,1174221,1174421,NM_001130045.1_2,+
53640,chr1,1174389,1174589,NM_001130045.1_3,+


In [41]:
all_3p_ExonIntronCoords_ToWrite.sort_values(by=["chrm","start"]).head(n=50)

,chrm,start,end,transcriptID,strand
63426,chr1,999332,999532,NM_001142467.1_1,-
63427,chr1,999513,999713,NM_001142467.1_2,-
53638,chr1,1174185,1174385,NM_001130045.1_1,+
53639,chr1,1174324,1174524,NM_001130045.1_2,+
53640,chr1,1179089,1179289,NM_001130045.1_3,+
53641,chr1,1179557,1179757,NM_001130045.1_4,+
53642,chr1,1179934,1180134,NM_001130045.1_5,+
53643,chr1,1180383,1180583,NM_001130045.1_6,+
53644,chr1,1180631,1180831,NM_001130045.1_7,+
53645,chr1,1181641,1181841,NM_001130045.1_8,+


In [52]:
all_3p_ExonIntronCoords_ToWrite_Sorted = pd.read_csv("../data/ExtractedExonIntronCoordinates_3p_FromGFFfile_ForSequence_Just3p.bed",sep="\t",header=None)
all_3p_ExonIntronCoords_ToWrite_Sorted.head()

,0,1,2,3,4
0,chr1,12513,12713,NR_046018.2_1,+
1,chr1,13121,13321,NR_046018.2_2,+
2,chr1,14729,14929,NR_024540.1_1,-
3,chr1,14938,15138,NR_024540.1_2,-
4,chr1,15847,16047,NR_024540.1_3,-


In [53]:
all_5p_ExonIntronCoords_ToWrite_Sorted = pd.read_csv("../data/ExtractedExonIntronCoordinates_5p_FromGFFfile_ForSequence_Just5p.bed",sep="\t",header=None)
all_5p_ExonIntronCoords_ToWrite_Sorted.head()

,0,1,2,3,4
0,chr1,12127,12327,NR_046018.2_1,+
1,chr1,12621,12821,NR_046018.2_2,+
2,chr1,14870,15070,NR_024540.1_1,-
3,chr1,15696,15896,NR_024540.1_2,-
4,chr1,16507,16707,NR_024540.1_3,-


In [54]:
all_5p_ExonIntronCoords_ToWrite_Sorted[all_5p_ExonIntronCoords_ToWrite_Sorted[3].str.contains("NR_046018.2")]

,0,1,2,3,4
0,chr1,12127,12327,NR_046018.2_1,+
1,chr1,12621,12821,NR_046018.2_2,+


In [55]:
%%script bash
bedtools getfasta -name -s -fo ../data/ExtractedExonIntronCoordinates_5p_FromGFFfile_ForSequence_Just5p.fa -fi /home/shared/GenomeReference/hg38.fa -bed ../data/ExtractedExonIntronCoordinates_5p_FromGFFfile_ForSequence_Just5p.bed
bedtools getfasta -name -s -fo ../data/ExtractedExonIntronCoordinates_3p_FromGFFfile_ForSequence_Just3p.fa -fi /home/shared/GenomeReference/hg38.fa -bed ../data/ExtractedExonIntronCoordinates_3p_FromGFFfile_ForSequence_Just3p.bed

In [56]:
# Let's get the unique coordinates 
all_5p_ExonIntronCoords_ToWrite_Sorted_unique = all_5p_ExonIntronCoords_ToWrite_Sorted.drop_duplicates(subset=[0,1,2])
print all_5p_ExonIntronCoords_ToWrite_Sorted.shape
print all_5p_ExonIntronCoords_ToWrite_Sorted_unique.shape
all_5p_ExonIntronCoords_ToWrite_Sorted_unique.head()

(1701899, 5)
(273789, 5)


,0,1,2,3,4
0,chr1,12127,12327,NR_046018.2_1,+
1,chr1,12621,12821,NR_046018.2_2,+
2,chr1,14870,15070,NR_024540.1_1,-
3,chr1,15696,15896,NR_024540.1_2,-
4,chr1,16507,16707,NR_024540.1_3,-


In [57]:
# Let's get the unique coordinates 
all_3p_ExonIntronCoords_ToWrite_Sorted_unique = all_3p_ExonIntronCoords_ToWrite_Sorted.drop_duplicates(subset=[0,1,2])
print all_3p_ExonIntronCoords_ToWrite_Sorted.shape
print all_3p_ExonIntronCoords_ToWrite_Sorted_unique.shape
all_3p_ExonIntronCoords_ToWrite_Sorted_unique.head()

(1701899, 5)
(265896, 5)


,0,1,2,3,4
0,chr1,12513,12713,NR_046018.2_1,+
1,chr1,13121,13321,NR_046018.2_2,+
2,chr1,14729,14929,NR_024540.1_1,-
3,chr1,14938,15138,NR_024540.1_2,-
4,chr1,15847,16047,NR_024540.1_3,-


In [58]:
# Write these coordinates to a file
all_5p_ExonIntronCoords_ToWrite_Sorted_unique.to_csv("../data/Unique_ExtractedExonIntronCoordinates_5p_FromGFFfile_ForSequence_Just5p.bed",sep="\t",header=False,index=False)
all_3p_ExonIntronCoords_ToWrite_Sorted_unique.to_csv("../data/Unique_ExtractedExonIntronCoordinates_3p_FromGFFfile_ForSequence_Just3p.bed",sep="\t",header=False,index=False)

In [59]:
%%script bash
bedtools getfasta -name -s -fo ../data/Unique_ExtractedExonIntronCoordinates_5p_FromGFFfile_ForSequence_Just5p.fa -fi /home/shared/GenomeReference/hg38.fa -bed ../data/Unique_ExtractedExonIntronCoordinates_5p_FromGFFfile_ForSequence_Just5p.bed
bedtools getfasta -name -s -fo ../data/Unique_ExtractedExonIntronCoordinates_3p_FromGFFfile_ForSequence_Just3p.fa -fi /home/shared/GenomeReference/hg38.fa -bed ../data/Unique_ExtractedExonIntronCoordinates_3p_FromGFFfile_ForSequence_Just3p.bed

## Let's get the intron coordinates from this exon data

In [ ]:
# Let's group by the transcript ID
data_exons_GrpedByFeature = data_exons.groupby(by=["transcriptID"])

In [ ]:
allintrons_DF = pd.DataFrame(columns=["chrm","start","end","strand","gene","transcriptID"])
for key, item in data_exons_GrpedByFeature:
    exons = data_exons_GrpedByFeature.get_group(key)
    # Only do this if there are more than 1 exon
    if exons.shape[0] > 1:
        clump_start_end = sorted(list(exons["start"].values) + list(exons["end"].values))
        # Get start coordinates of introns 
        start_coords_intron = [clump_start_end[i] for i in range(1,len(clump_start_end)-2,2)]
        # Get end coordinates of introns
        end_coords_intron = [clump_start_end[i]-1 for i in range(2,len(clump_start_end)-1,2)]
        if len(start_coords_intron)==len(end_coords_intron):
            chrm = [list(exons["chrm"].values)[0]]*len(start_coords_intron)
            gene = [list(exons["gene"].values)[0]]*len(start_coords_intron)
            strand = [list(exons["strand"].values)[0]]*len(start_coords_intron)
            transcriptID = [key]*len(start_coords_intron)
            newintron_DF = pd.DataFrame({"chrm":chrm,"start":start_coords_intron,"end":end_coords_intron,"strand":strand,"transcriptID":transcriptID,"gene":gene},columns=["chrm","start","end","strand","gene","transcriptID"])
            allintrons_DF = pd.concat([allintrons_DF,newintron_DF],ignore_index=True,axis=0)
            allintrons_DF = allintrons_DF.reset_index(drop=True)
        else:
            print key

In [ ]:
allintrons_DF.head()

In [ ]:
allintrons_DF.to_csv("../data/ExtractedIntronCoordinatesFromGFFfile.bed",sep="\t",header=False,index=False)

In [ ]:
allintrons_DF = pd.read_csv("../data/ExtractedIntronCoordinatesFromGFFfile.bed",sep="\t",header=None)
print allintrons_DF.shape
allintrons_DF.head()

In [ ]:
allintrons_DF_groupedByTranscript = allintrons_DF.groupby(by=[5])

In [ ]:
allintrons_DF_groupedByTranscript.get_group("NM_000014.5")

In [ ]:
allIntronicCoords_ToWrite = pd.DataFrame(columns=["chrm","start","end","transcriptID","strand"])
new_geneNameDF = pd.DataFrame(columns=["TranscriptID","Gene"])
for key, item in allintrons_DF_groupedByTranscript:
    introns = allintrons_DF_groupedByTranscript.get_group(key)
    introns_sorted = introns.sort_values(by=[0,1])
    num_rows = introns.shape[0]
    transcriptIDs = [key+"_"+str(i+1) for i in range(num_rows)]
    new_DF = pd.DataFrame({"chrm":introns_sorted[0],"start":introns_sorted[1],"end":introns_sorted[2],"transcriptID":transcriptIDs,"strand":introns_sorted[3]},columns=["chrm","start","end","transcriptID","strand"])
    allIntronicCoords_ToWrite = pd.concat([allIntronicCoords_ToWrite,new_DF],ignore_index=True,axis=0)
    allIntronicCoords_ToWrite.reset_index(drop=True)
    new_geneNameDF = pd.concat([new_geneNameDF,pd.DataFrame({"Gene":introns_sorted[4],"TranscriptID":transcriptIDs},columns=["TranscriptID","Gene"])],ignore_index=True,axis=0)
    new_geneNameDF.reset_index(drop=True)

In [ ]:
allIntronicCoords_ToWrite = pd.read_csv("../data/ExtractedIntronCoordinatesFromGFFfile_ForSequence.bed",sep="\t",header=None)
allIntronicCoords_ToWrite.head()

In [ ]:
new_geneNameDF = pd.read_csv("../data/JustIntrons_GeneNameTranscriptID_NCBI_RefSeq_hg38_FromGFFfile.tsv",sep="\t",header=None)
new_geneNameDF.head()

In [ ]:
%%script bash
bedtools getfasta -name -s -fo ../data/ExtractedIntronCoordinatesFromGFFfile_sequences.fa -fi /home/shared/GenomeReference/hg38.fa -bed ../data/ExtractedIntronCoordinatesFromGFFfile_ForSequence.bed

## Let's get the genomic coordinates of transcripts from this exon data

In [ ]:
# Let's extract the entire genomic coordinate
allGenomicCoords_DF = pd.DataFrame(columns=["chrm","start","end","strand","gene","transcriptID"])
for key, item in data_exons_GrpedByFeature:
    exons = data_exons_GrpedByFeature.get_group(key)
    chrm = [list(exons["chrm"].values)[0]]
    gene = [list(exons["gene"].values)[0]]
    strand = [list(exons["strand"].values)[0]]
    transcriptID = [key]
    clump_start_end = sorted(list(exons["start"].values) + list(exons["end"].values))
    start = [clump_start_end[0]]
    end = [clump_start_end[len(clump_start_end)-1]]
    # Only do this if there are more than 1 exon
    newGenomicCoord_DF = pd.DataFrame({"chrm":chrm,"start":start,"end":end,"strand":strand,"transcriptID":transcriptID,"gene":gene},columns=["chrm","start","end","strand","gene","transcriptID"])
    allGenomicCoords_DF = pd.concat([allGenomicCoords_DF,newGenomicCoord_DF],ignore_index=True,axis=0)
    allGenomicCoords_DF = allGenomicCoords_DF.reset_index(drop=True)

In [ ]:
allGenomicCoords_DF.to_csv("../data/ExtractedGenomicCoordinatesFromGFFfile.bed",sep="\t",header=False,index=False)

In [ ]:
allGenomicCoords_DF = pd.read_csv("../data/ExtractedGenomicCoordinatesFromGFFfile.bed",sep="\t",header=None)
print allGenomicCoords_DF.shape
allGenomicCoords_DF.head()

In [ ]:
allGenomicCoords_DF_sorted = allGenomicCoords_DF.sort_values(by=[0,1])
allGenomicCoords_DF_sorted.head()

In [ ]:
allGenomicCoords_DF_sorted[0] = allGenomicCoords_DF_sorted[0].replace("chr23","chrX")
allGenomicCoords_DF_sorted[0] = allGenomicCoords_DF_sorted[0].replace("chr24","chrY")

In [ ]:
allGenomicCoords_DF_sorted[[0,1,2,5,3]].to_csv("../data/ExtractedGenomicCoordinatesFromGFFfile_Sorted.bed",sep="\t",header=False,index=False)

In [ ]:
%%script bash
bedtools getfasta -name -s -fo ../data/ExtractedGenomicCoordinatesFromGFFfile_sequences.fa -fi /home/shared/GenomeReference/hg38.fa -bed ../data/ExtractedGenomicCoordinatesFromGFFfile_Sorted.bed

## Let's get the coordinates and gene ID names to make simplified annotation format (SAF) for featureCounts

### First for transcriptome coordinates, we need both geneID and transcriptID

In [19]:
data_genes = pd.read_csv("../data/GRCh38_latest_genomic_ValidChroms_OnlyGenes.bed",sep="\t",header=None)
print data_genes.shape
data_genes.head()

(2017619, 8)


,0,1,2,3,4,5,6,7
0,chr1,11874,12227,+,DDX11L1,NR_046018.2,BestRefSeq,exon
1,chr1,11874,14409,+,DDX11L1,NR_046018.2,BestRefSeq,transcript
2,chr1,12613,12721,+,DDX11L1,NR_046018.2,BestRefSeq,exon
3,chr1,13221,14409,+,DDX11L1,NR_046018.2,BestRefSeq,exon
4,chr1,14362,14829,-,WASH7P,NR_024540.1,BestRefSeq,exon


In [20]:
# Set column names
data_genes.columns = ["chrm","start","end","strand","gene","transcriptID","RefSeqType","feature"]

In [21]:
# Only grab exons 
data_exons = data_genes[data_genes["feature"]=="exon"]
print data_exons.shape
data_exons.head()

(1859728, 8)


,chrm,start,end,strand,gene,transcriptID,RefSeqType,feature
0,chr1,11874,12227,+,DDX11L1,NR_046018.2,BestRefSeq,exon
2,chr1,12613,12721,+,DDX11L1,NR_046018.2,BestRefSeq,exon
3,chr1,13221,14409,+,DDX11L1,NR_046018.2,BestRefSeq,exon
4,chr1,14362,14829,-,WASH7P,NR_024540.1,BestRefSeq,exon
6,chr1,14970,15038,-,WASH7P,NR_024540.1,BestRefSeq,exon


In [22]:
# Let's write a file that is SAF for just transcriptIDs
data_exons_justTranscriptIDs = data_exons[["transcriptID","chrm","start","end","strand"]]
data_exons_justTranscriptIDs.columns = ["GeneID","Chr","Start","End","Strand"]
data_exons_justTranscriptIDs.head()

,GeneID,Chr,Start,End,Strand
0,NR_046018.2,chr1,11874,12227,+
2,NR_046018.2,chr1,12613,12721,+
3,NR_046018.2,chr1,13221,14409,+
4,NR_024540.1,chr1,14362,14829,-
6,NR_024540.1,chr1,14970,15038,-


In [10]:
data_exons_justTranscriptIDs.to_csv("../data/SAFformat_GRCh38_latest_genomic_ValidChroms_OnlyTranscripts.bed",sep="\t",header=True,index=False)

In [ ]:
# Get 

In [9]:
# Let's write a file that is SAF for just genes
data_exons_justGeneIDs = data_exons[["gene","chrm","start","end","strand"]]
data_exons_justGeneIDs.columns = ["GeneID","Chr","Start","End","Strand"]
data_exons_justGeneIDs.head()

,GeneID,Chr,Start,End,Strand
0,DDX11L1,chr1,11874,12227,+
2,DDX11L1,chr1,12613,12721,+
3,DDX11L1,chr1,13221,14409,+
4,WASH7P,chr1,14362,14829,-
6,WASH7P,chr1,14970,15038,-


In [11]:
data_exons_justGeneIDs.to_csv("../data/SAFformat_GRCh38_latest_genomic_ValidChroms_OnlyGenes.bed",sep="\t",header=True,index=False)

### Next for genomic coordinates

In [44]:
### First for genome coordinates, we need both geneID and transcriptID
data_genes_transcripts = pd.read_csv("../data/GeneNameTranscriptID_NCBI_RefSeq_hg38_FromGFFfile.tsv",sep="\t",header=None)
print data_genes_transcripts.shape
data_genes_transcripts.head()

(159998, 2)


,0,1
0,NR_046018.2,DDX11L1
1,NR_024540.1,WASH7P
2,NR_106918.1,MIR6859-1
3,XR_001737835.1,MIR1302-2HG
4,NR_036051.1,MIR1302-2


In [43]:
coords_genomic = pd.read_csv("../data/ExtractedGenomicCoordinatesFromGFFfile_Sorted.bed",sep="\t",header=None)
print coords_genomic.shape
coords_genomic.head()

(157829, 5)


,0,1,2,3,4
0,chr1,11874,14409,NR_046018.2,+
1,chr1,14362,29370,NR_024540.1,-
2,chr1,17369,17436,NR_106918.1,-
3,chr1,29926,31295,XR_001737835.1,+
4,chr1,30366,30503,NR_036051.1,+


In [46]:
merged_coords_genomic = coords_genomic.merge(data_genes_transcripts,right_on=0,left_on=3)
print merged_coords_genomic.shape
merged_coords_genomic.head()

(157829, 7)


,0_x,1_x,2,3,4,0_y,1_y
0,chr1,11874,14409,NR_046018.2,+,NR_046018.2,DDX11L1
1,chr1,14362,29370,NR_024540.1,-,NR_024540.1,WASH7P
2,chr1,17369,17436,NR_106918.1,-,NR_106918.1,MIR6859-1
3,chr1,29926,31295,XR_001737835.1,+,XR_001737835.1,MIR1302-2HG
4,chr1,30366,30503,NR_036051.1,+,NR_036051.1,MIR1302-2


In [48]:
# Get the SAF format
data_genes_justGeneIDs = merged_coords_genomic.iloc[:,[6,0,1,2,4]]
print data_genes_justGeneIDs.shape
data_genes_justGeneIDs.head()

(157829, 5)


,1_y,0_x,1_x,2,4
0,DDX11L1,chr1,11874,14409,+
1,WASH7P,chr1,14362,29370,-
2,MIR6859-1,chr1,17369,17436,-
3,MIR1302-2HG,chr1,29926,31295,+
4,MIR1302-2,chr1,30366,30503,+


In [49]:
data_genes_justGeneIDs.columns = ["GeneID","Chr","Start","End","Strand"]
data_genes_justGeneIDs.head()

,GeneID,Chr,Start,End,Strand
0,DDX11L1,chr1,11874,14409,+
1,WASH7P,chr1,14362,29370,-
2,MIR6859-1,chr1,17369,17436,-
3,MIR1302-2HG,chr1,29926,31295,+
4,MIR1302-2,chr1,30366,30503,+


In [50]:
data_genes_justGeneIDs_sorted = data_genes_justGeneIDs.sort_values(by=["Chr","Start","End"])
data_genes_justGeneIDs_sorted.head()

,GeneID,Chr,Start,End,Strand
0,DDX11L1,chr1,11874,14409,+
1,WASH7P,chr1,14362,29370,-
2,MIR6859-1,chr1,17369,17436,-
3,MIR1302-2HG,chr1,29926,31295,+
4,MIR1302-2,chr1,30366,30503,+


In [51]:
data_genes_justGeneIDs_sorted.to_csv("../data/SAFformat_ExtractedGenomicCoordinatesFromGFFfile_OnlyGenes.bed",sep="\t",header=True,index=False)

### Next for exon-intron coordinates

In [2]:
### First for exon-intron coordinates, we need both geneID and transcriptID
data_transcripts = pd.read_csv("../data/ExtractedExonIntronCoordinates_5p_FromGFFfile_ForSequence_Just5p.bed",sep="\t",header=None)
print data_transcripts.shape
data_transcripts.head()

(1701899, 5)


,0,1,2,3,4
0,chr1,12127,12327,NR_046018.2_1,+
1,chr1,12621,12821,NR_046018.2_2,+
2,chr1,14870,15070,NR_024540.1_1,-
3,chr1,15696,15896,NR_024540.1_2,-
4,chr1,16507,16707,NR_024540.1_3,-


In [3]:
### First for exon-intron coordinates, we need both geneID and transcriptID
data_transcriptsToGenes = pd.read_csv("../data/ExonIntronCoords_GeneNameTranscriptID_NCBI_RefSeq_hg38_FromGFFfile_5p.tsv",sep="\t",header=None)
print data_transcriptsToGenes.shape
data_transcriptsToGenes.head()

(1701899, 2)


,0,1
0,NM_000014.5_1,A2M
1,NM_000014.5_2,A2M
2,NM_000014.5_3,A2M
3,NM_000014.5_4,A2M
4,NM_000014.5_5,A2M


In [4]:
# Merge the two data sets
data_all_5p = data_transcripts.merge(data_transcriptsToGenes,left_on=3,right_on=0)
print data_all_5p.shape
data_all_5p.head()

(1701899, 7)


,0_x,1_x,2,3,4,0_y,1_y
0,chr1,12127,12327,NR_046018.2_1,+,NR_046018.2_1,DDX11L1
1,chr1,12621,12821,NR_046018.2_2,+,NR_046018.2_2,DDX11L1
2,chr1,14870,15070,NR_024540.1_1,-,NR_024540.1_1,WASH7P
3,chr1,15696,15896,NR_024540.1_2,-,NR_024540.1_2,WASH7P
4,chr1,16507,16707,NR_024540.1_3,-,NR_024540.1_3,WASH7P


In [5]:
# Get the SAF format
data_all_5p_justGeneIDs = data_all_5p.iloc[:,[6,0,1,2,4]]
print data_all_5p_justGeneIDs.shape
data_all_5p_justGeneIDs.head()

(1701899, 5)


,1_y,0_x,1_x,2,4
0,DDX11L1,chr1,12127,12327,+
1,DDX11L1,chr1,12621,12821,+
2,WASH7P,chr1,14870,15070,-
3,WASH7P,chr1,15696,15896,-
4,WASH7P,chr1,16507,16707,-


In [6]:
data_all_5p_justGeneIDs.columns = ["GeneID","Chr","Start","End","Strand"]
data_all_5p_justGeneIDs.head()

,GeneID,Chr,Start,End,Strand
0,DDX11L1,chr1,12127,12327,+
1,DDX11L1,chr1,12621,12821,+
2,WASH7P,chr1,14870,15070,-
3,WASH7P,chr1,15696,15896,-
4,WASH7P,chr1,16507,16707,-


In [66]:
data_all_5p_justGeneIDs.to_csv("../data/SAFformat_ExtractedExonIntronCoordinatesFromGFFfile_5p_OnlyGenes.bed",sep="\t",header=True,index=False)

In [7]:
# Get the SAF format for transcriptID
data_all_5p_justTranscriptIDs = data_all_5p.iloc[:,[3,0,1,2,4]]
print data_all_5p_justTranscriptIDs.shape
data_all_5p_justTranscriptIDs.head()

(1701899, 5)


,3,0_x,1_x,2,4
0,NR_046018.2_1,chr1,12127,12327,+
1,NR_046018.2_2,chr1,12621,12821,+
2,NR_024540.1_1,chr1,14870,15070,-
3,NR_024540.1_2,chr1,15696,15896,-
4,NR_024540.1_3,chr1,16507,16707,-


In [8]:
data_all_5p_justTranscriptIDs.columns = ["GeneID","Chr","Start","End","Strand"]
data_all_5p_justTranscriptIDs.head()

,GeneID,Chr,Start,End,Strand
0,NR_046018.2_1,chr1,12127,12327,+
1,NR_046018.2_2,chr1,12621,12821,+
2,NR_024540.1_1,chr1,14870,15070,-
3,NR_024540.1_2,chr1,15696,15896,-
4,NR_024540.1_3,chr1,16507,16707,-


In [9]:
data_all_5p_justTranscriptIDs.to_csv("../data/SAFformat_ExtractedExonIntronCoordinatesFromGFFfile_5p_OnlyTranscriptIDs.bed",sep="\t",header=True,index=False)

In [10]:
### First for exon-intron coordinates, we need both geneID and transcriptID
data_transcripts = pd.read_csv("../data/ExtractedExonIntronCoordinates_3p_FromGFFfile_ForSequence_Just3p.bed",sep="\t",header=None)
print data_transcripts.shape
data_transcripts.head()

(1701899, 5)


,0,1,2,3,4
0,chr1,12513,12713,NR_046018.2_1,+
1,chr1,13121,13321,NR_046018.2_2,+
2,chr1,14729,14929,NR_024540.1_1,-
3,chr1,14938,15138,NR_024540.1_2,-
4,chr1,15847,16047,NR_024540.1_3,-


In [11]:
### First for exon-intron coordinates, we need both geneID and transcriptID
data_transcriptsToGenes = pd.read_csv("../data/ExonIntronCoords_GeneNameTranscriptID_NCBI_RefSeq_hg38_FromGFFfile_3p.tsv",sep="\t",header=None)
print data_transcriptsToGenes.shape
data_transcriptsToGenes.head()

(1701899, 2)


,0,1
0,NM_000014.5_1,A2M
1,NM_000014.5_2,A2M
2,NM_000014.5_3,A2M
3,NM_000014.5_4,A2M
4,NM_000014.5_5,A2M


In [12]:
# Merge the two data sets
data_all_3p = data_transcripts.merge(data_transcriptsToGenes,left_on=3,right_on=0)
print data_all_3p.shape
data_all_3p.head()

(1701899, 7)


,0_x,1_x,2,3,4,0_y,1_y
0,chr1,12513,12713,NR_046018.2_1,+,NR_046018.2_1,DDX11L1
1,chr1,13121,13321,NR_046018.2_2,+,NR_046018.2_2,DDX11L1
2,chr1,14729,14929,NR_024540.1_1,-,NR_024540.1_1,WASH7P
3,chr1,14938,15138,NR_024540.1_2,-,NR_024540.1_2,WASH7P
4,chr1,15847,16047,NR_024540.1_3,-,NR_024540.1_3,WASH7P


In [13]:
# Get the SAF format
data_all_3p_justGeneIDs = data_all_3p.iloc[:,[6,0,1,2,4]]
print data_all_3p_justGeneIDs.shape
data_all_3p_justGeneIDs.head()

(1701899, 5)


,1_y,0_x,1_x,2,4
0,DDX11L1,chr1,12513,12713,+
1,DDX11L1,chr1,13121,13321,+
2,WASH7P,chr1,14729,14929,-
3,WASH7P,chr1,14938,15138,-
4,WASH7P,chr1,15847,16047,-


In [14]:
data_all_3p_justGeneIDs.columns = ["GeneID","Chr","Start","End","Strand"]
data_all_3p_justGeneIDs.head()

,GeneID,Chr,Start,End,Strand
0,DDX11L1,chr1,12513,12713,+
1,DDX11L1,chr1,13121,13321,+
2,WASH7P,chr1,14729,14929,-
3,WASH7P,chr1,14938,15138,-
4,WASH7P,chr1,15847,16047,-


In [72]:
data_all_3p_justGeneIDs.to_csv("../data/SAFformat_ExtractedExonIntronCoordinatesFromGFFfile_3p_OnlyGenes.bed",sep="\t",header=True,index=False)

In [16]:
# Get the SAF format
data_all_3p_justTranscriptIDs = data_all_3p.iloc[:,[3,0,1,2,4]]
print data_all_3p_justTranscriptIDs.shape
data_all_3p_justTranscriptIDs.head()

(1701899, 5)


,3,0_x,1_x,2,4
0,NR_046018.2_1,chr1,12513,12713,+
1,NR_046018.2_2,chr1,13121,13321,+
2,NR_024540.1_1,chr1,14729,14929,-
3,NR_024540.1_2,chr1,14938,15138,-
4,NR_024540.1_3,chr1,15847,16047,-


In [17]:
data_all_3p_justTranscriptIDs.columns = ["GeneID","Chr","Start","End","Strand"]
data_all_3p_justTranscriptIDs.head()

,GeneID,Chr,Start,End,Strand
0,NR_046018.2_1,chr1,12513,12713,+
1,NR_046018.2_2,chr1,13121,13321,+
2,NR_024540.1_1,chr1,14729,14929,-
3,NR_024540.1_2,chr1,14938,15138,-
4,NR_024540.1_3,chr1,15847,16047,-


In [18]:
data_all_3p_justTranscriptIDs.to_csv("../data/SAFformat_ExtractedExonIntronCoordinatesFromGFFfile_3p_OnlyTranscriptIDs.bed",sep="\t",header=True,index=False)

### Let's get exon-exon coordinates

In [36]:
### First for exon-intron coordinates, we need both geneID and transcriptID
data_transcripts = pd.read_csv("../data/ExtractedExonIntronCoordinates_5p_FromGFFfile_ForSequence_Just5p.bed",sep="\t",header=None)
print data_transcripts.shape
data_transcripts.head()

(1701899, 5)


,0,1,2,3,4
0,chr1,12127,12327,NR_046018.2_1,+
1,chr1,12621,12821,NR_046018.2_2,+
2,chr1,14870,15070,NR_024540.1_1,-
3,chr1,15696,15896,NR_024540.1_2,-
4,chr1,16507,16707,NR_024540.1_3,-


In [37]:
# Separate into plus and minus
data_transcripts_plus = data_transcripts[data_transcripts[4]=="+"]
data_transcripts_minus = data_transcripts[data_transcripts[4]=="-"]

In [38]:
data_transcripts_exonexon = pd.DataFrame({"chrom":pd.concat([data_transcripts_plus[0],data_transcripts_minus[0]]),
                                         "start":pd.concat([data_transcripts_plus[1],data_transcripts_minus[1]+50]),
                                          "end":pd.concat([data_transcripts_plus[1]+50,data_transcripts_minus[2]]),
                                          "transcriptID":pd.concat([data_transcripts_plus[3],data_transcripts_minus[3]]),
                                          "strand":pd.concat([data_transcripts_plus[4],data_transcripts_minus[4]])},
                                        columns=["chrom","start","end","transcriptID","strand"])
data_transcripts_exonexon.head()

,chrom,start,end,transcriptID,strand
0,chr1,12127,12177,NR_046018.2_1,+
1,chr1,12621,12671,NR_046018.2_2,+
12,chr1,29939,29989,XR_001737835.1_1,+
13,chr1,30567,30617,XR_001737835.1_2,+
75,chr1,182646,182696,NR_148357.1_1,+


In [48]:
data_transcripts_exonintron = pd.DataFrame({"chrom":pd.concat([data_transcripts_plus[0],data_transcripts_minus[0]]),
                                         "start":pd.concat([data_transcripts_plus[1]+50,data_transcripts_minus[1]+50]),
                                          "end":pd.concat([data_transcripts_plus[2]-50,data_transcripts_minus[2]-50]),
                                          "transcriptID":pd.concat([data_transcripts_plus[3],data_transcripts_minus[3]]),
                                          "strand":pd.concat([data_transcripts_plus[4],data_transcripts_minus[4]])},
                                        columns=["chrom","start","end","transcriptID","strand"])
data_transcripts_exonintron.head()

,chrom,start,end,transcriptID,strand
0,chr1,12563,12663,NR_046018.2_1,+
1,chr1,13171,13271,NR_046018.2_2,+
12,chr1,30514,30614,XR_001737835.1_1,+
13,chr1,30926,31026,XR_001737835.1_2,+
76,chr1,183082,183182,NR_148357.1_1,+


In [49]:
data_transcripts_exonexon_sorted = data_transcripts_exonexon.sort_values(by=["chrom","start","end"])
data_transcripts_exonexon_sorted.head()

,chrom,start,end,transcriptID,strand
0,chr1,12563,12713,NR_046018.2_1,+
1,chr1,13171,13321,NR_046018.2_2,+
2,chr1,14729,14779,NR_024540.1_1,-
3,chr1,14938,14988,NR_024540.1_2,-
4,chr1,15847,15897,NR_024540.1_3,-


In [50]:
data_transcripts_exonintron_sorted = data_transcripts_exonintron.sort_values(by=["chrom","start","end"])
data_transcripts_exonintron_sorted.head()

,chrom,start,end,transcriptID,strand
0,chr1,12563,12663,NR_046018.2_1,+
1,chr1,13171,13271,NR_046018.2_2,+
2,chr1,14779,14879,NR_024540.1_1,-
3,chr1,14988,15088,NR_024540.1_2,-
4,chr1,15897,15997,NR_024540.1_3,-


In [53]:
data_all_5p_justTranscriptIDs = data_transcripts_exonexon_sorted.iloc[:,[3,0,1,2,4]]
data_all_5p_justTranscriptIDs.columns = ["GeneID","Chr","Start","End","Strand"]
data_all_5p_justTranscriptIDs.head()

,GeneID,Chr,Start,End,Strand
0,NR_046018.2_1,chr1,12563,12713,+
1,NR_046018.2_2,chr1,13171,13321,+
2,NR_024540.1_1,chr1,14729,14779,-
3,NR_024540.1_2,chr1,14938,14988,-
4,NR_024540.1_3,chr1,15847,15897,-


In [54]:
data_all_5p_justTranscriptIDs.to_csv("../data/SAFformat_ExtractedExonExonCoordinatesFromGFFfile_5p_OnlyTranscriptIDs.bed",sep="\t",header=True,index=False)

In [55]:
data_all_5p_justTranscriptIDs = data_transcripts_exonintron_sorted.iloc[:,[3,0,1,2,4]]
data_all_5p_justTranscriptIDs.columns = ["GeneID","Chr","Start","End","Strand"]
data_all_5p_justTranscriptIDs.head()

,GeneID,Chr,Start,End,Strand
0,NR_046018.2_1,chr1,12563,12663,+
1,NR_046018.2_2,chr1,13171,13271,+
2,NR_024540.1_1,chr1,14779,14879,-
3,NR_024540.1_2,chr1,14988,15088,-
4,NR_024540.1_3,chr1,15897,15997,-


In [56]:
data_all_5p_justTranscriptIDs.to_csv("../data/SAFformat_ExtractedIntronExonCoordinatesFromGFFfile_5p_OnlyTranscriptIDs-100bp.bed",sep="\t",header=True,index=False)

In [42]:
### First for exon-intron coordinates, we need both geneID and transcriptID
data_transcripts = pd.read_csv("../data/ExtractedExonIntronCoordinates_3p_FromGFFfile_ForSequence_Just3p.bed",sep="\t",header=None)
print data_transcripts.shape
data_transcripts.head()

(1701899, 5)


,0,1,2,3,4
0,chr1,12513,12713,NR_046018.2_1,+
1,chr1,13121,13321,NR_046018.2_2,+
2,chr1,14729,14929,NR_024540.1_1,-
3,chr1,14938,15138,NR_024540.1_2,-
4,chr1,15847,16047,NR_024540.1_3,-


In [43]:
# Separate into plus and minus
data_transcripts_plus = data_transcripts[data_transcripts[4]=="+"]
data_transcripts_minus = data_transcripts[data_transcripts[4]=="-"]

In [44]:
data_transcripts_exonexon = pd.DataFrame({"chrom":pd.concat([data_transcripts_plus[0],data_transcripts_minus[0]]),
                                         "start":pd.concat([data_transcripts_plus[1]+50,data_transcripts_minus[1]]),
                                          "end":pd.concat([data_transcripts_plus[2],data_transcripts_minus[1]+50]),
                                          "transcriptID":pd.concat([data_transcripts_plus[3],data_transcripts_minus[3]]),
                                          "strand":pd.concat([data_transcripts_plus[4],data_transcripts_minus[4]])},
                                        columns=["chrom","start","end","transcriptID","strand"])
data_transcripts_exonexon.head()

,chrom,start,end,transcriptID,strand
0,chr1,12563,12713,NR_046018.2_1,+
1,chr1,13171,13321,NR_046018.2_2,+
12,chr1,30514,30664,XR_001737835.1_1,+
13,chr1,30926,31076,XR_001737835.1_2,+
76,chr1,183082,183232,NR_148357.1_1,+


In [45]:
data_transcripts_exonexon_sorted = data_transcripts_exonexon.sort_values(by=["chrom","start","end"])
data_transcripts_exonexon_sorted.head()

,chrom,start,end,transcriptID,strand
0,chr1,12563,12713,NR_046018.2_1,+
1,chr1,13171,13321,NR_046018.2_2,+
2,chr1,14729,14779,NR_024540.1_1,-
3,chr1,14938,14988,NR_024540.1_2,-
4,chr1,15847,15897,NR_024540.1_3,-


In [46]:
data_all_3p_justTranscriptIDs = data_transcripts_exonexon_sorted.iloc[:,[3,0,1,2,4]]
data_all_3p_justTranscriptIDs.columns = ["GeneID","Chr","Start","End","Strand"]
data_all_3p_justTranscriptIDs.head()

,GeneID,Chr,Start,End,Strand
0,NR_046018.2_1,chr1,12563,12713,+
1,NR_046018.2_2,chr1,13171,13321,+
2,NR_024540.1_1,chr1,14729,14779,-
3,NR_024540.1_2,chr1,14938,14988,-
4,NR_024540.1_3,chr1,15847,15897,-


In [47]:
data_all_3p_justTranscriptIDs.to_csv("../data/SAFformat_ExtractedExonExonCoordinatesFromGFFfile_3p_OnlyTranscriptIDs.bed",sep="\t",header=True,index=False)

### Get coordinates of introns that are less than 100 bases

In [94]:
intron_coords = pd.read_csv("../data/ExtractedIntronCoordinatesFromGFFfile_NoOverlapExon.bed",sep="\t",header=None)
print(intron_coords.shape)
intron_coords.head()

(1316596, 5)


,0,1,2,3,4
0,chr1,12227,12612,NR_046018.2_1,+
1,chr1,12721,13220,NR_046018.2_2,+
2,chr1,14829,14969,NR_024540.1_1,-
3,chr1,15038,15795,NR_024540.1_2,-
4,chr1,15947,16606,NR_024540.1_3,-


In [95]:
# Get introns that are less than 100 bases
introns_lessthan100 = intron_coords[intron_coords[2]-intron_coords[1]<=100]
print(introns_lessthan100.shape)
introns_lessthan100.head()

(63714, 5)


,0,1,2,3,4
5,chr1,16765,16857,NR_024540.1_4,-
23,chr1,139696,139789,NR_039983.2_1,-
54,chr1,187287,187379,XR_001737556.1_4,-
70,chr1,358183,358283,XR_002958507.1_2,-
79,chr1,494898,494991,NR_028322.1_1,-


In [96]:
introns_lessthan100.columns= ["chrom","start","end","transcriptID","strand"]

In [97]:
# Get introns that are greater than 100 bases
introns_greaterthan100 = intron_coords[intron_coords[2]-intron_coords[1]>100]
print(introns_greaterthan100.shape)
introns_greaterthan100.head()

(1252882, 5)


,0,1,2,3,4
0,chr1,12227,12612,NR_046018.2_1,+
1,chr1,12721,13220,NR_046018.2_2,+
2,chr1,14829,14969,NR_024540.1_1,-
3,chr1,15038,15795,NR_024540.1_2,-
4,chr1,15947,16606,NR_024540.1_3,-


In [98]:
# Split up introns that are plus and minus strand
introns_greaterthan100_plus = introns_greaterthan100[introns_greaterthan100[4]=="+"]
introns_greaterthan100_minus = introns_greaterthan100[introns_greaterthan100[4]=="-"]

In [99]:
# Get 5' ends of plus and minus
introns_greaterthan100_5p = pd.DataFrame({"chrom":pd.concat([introns_greaterthan100_plus[0],introns_greaterthan100_minus[0]]),
                                         "start":pd.concat([introns_greaterthan100_plus[1],introns_greaterthan100_minus[2]-100]),
                                          "end":pd.concat([introns_greaterthan100_plus[1]+100,introns_greaterthan100_minus[2]]),
                                          "transcriptID":pd.concat([introns_greaterthan100_plus[3],introns_greaterthan100_minus[3]]),
                                          "strand":pd.concat([introns_greaterthan100_plus[4],introns_greaterthan100_minus[4]])},
                                        columns=["chrom","start","end","transcriptID","strand"])

In [100]:
print(introns_greaterthan100_5p.shape)
introns_greaterthan100_5p.head()

(1252882, 5)


,chrom,start,end,transcriptID,strand
0,chr1,12227,12327,NR_046018.2_1,+
1,chr1,12721,12821,NR_046018.2_2,+
11,chr1,30667,30767,XR_001737835.1_2,+
49,chr1,182746,182846,NR_148357.1_1,+
50,chr1,183240,183340,NR_148357.1_2,+


In [101]:
# concantenate the 5p with introns less than 100
#introns_5p = pd.concat([introns_lessthan100,introns_greaterthan100_5p],axis=0)
introns_5p = introns_greaterthan100_5p
print(introns_5p.shape)
introns_5p.head()

(1252882, 5)


,chrom,start,end,transcriptID,strand
0,chr1,12227,12327,NR_046018.2_1,+
1,chr1,12721,12821,NR_046018.2_2,+
11,chr1,30667,30767,XR_001737835.1_2,+
49,chr1,182746,182846,NR_148357.1_1,+
50,chr1,183240,183340,NR_148357.1_2,+


In [102]:
# Sort introns 
introns_5p_sorted = introns_5p.sort_values(by=["chrom","start","end"])
introns_5p_sorted.head()

,chrom,start,end,transcriptID,strand
0,chr1,12227,12327,NR_046018.2_1,+
1,chr1,12721,12821,NR_046018.2_2,+
2,chr1,14869,14969,NR_024540.1_1,-
3,chr1,15695,15795,NR_024540.1_2,-
4,chr1,16506,16606,NR_024540.1_3,-


In [103]:
introns_5p_sorted_justTranscriptIDs = introns_5p_sorted.iloc[:,[3,0,1,2,4]]
introns_5p_sorted_justTranscriptIDs.columns = ["GeneID","Chr","Start","End","Strand"]
introns_5p_sorted_justTranscriptIDs.head()

,GeneID,Chr,Start,End,Strand
0,NR_046018.2_1,chr1,12227,12327,+
1,NR_046018.2_2,chr1,12721,12821,+
2,NR_024540.1_1,chr1,14869,14969,-
3,NR_024540.1_2,chr1,15695,15795,-
4,NR_024540.1_3,chr1,16506,16606,-


In [111]:
introns_5p_sorted_justTranscriptIDs["Start"] = introns_5p_sorted_justTranscriptIDs["Start"] + 1

In [112]:
introns_5p_sorted_justTranscriptIDs.head()

,GeneID,Chr,Start,End,Strand
0,NR_046018.2_1,chr1,12228,12327,+
1,NR_046018.2_2,chr1,12722,12821,+
2,NR_024540.1_1,chr1,14870,14969,-
3,NR_024540.1_2,chr1,15696,15795,-
4,NR_024540.1_3,chr1,16507,16606,-


In [113]:
introns_5p_sorted_justTranscriptIDs.to_csv("../data/SAFformat_ExtractedIntronCoordinatesFromGFFfile_5p_OnlyTranscriptIDs.bed",sep="\t",header=True,index=False)

In [114]:
# Get 3' ends of plus and minus
introns_greaterthan100_3p = pd.DataFrame({"chrom":pd.concat([introns_greaterthan100_plus[0],introns_greaterthan100_minus[0]]),
                                         "start":pd.concat([introns_greaterthan100_plus[2]-100,introns_greaterthan100_minus[1]]),
                                          "end":pd.concat([introns_greaterthan100_plus[2],introns_greaterthan100_minus[1]+100]),
                                          "transcriptID":pd.concat([introns_greaterthan100_plus[3],introns_greaterthan100_minus[3]]),
                                          "strand":pd.concat([introns_greaterthan100_plus[4],introns_greaterthan100_minus[4]])},
                                        columns=["chrom","start","end","transcriptID","strand"])

In [115]:
print(introns_greaterthan100_3p.shape)
introns_greaterthan100_3p.head()

(1252882, 5)


,chrom,start,end,transcriptID,strand
0,chr1,12512,12612,NR_046018.2_1,+
1,chr1,13120,13220,NR_046018.2_2,+
11,chr1,30875,30975,XR_001737835.1_2,+
49,chr1,183031,183131,NR_148357.1_1,+
50,chr1,183639,183739,NR_148357.1_2,+


In [116]:
# concantenate the 3p with introns less than 100
#introns_3p = pd.concat([introns_lessthan100,introns_greaterthan100_3p],axis=0)
introns_3p = introns_greaterthan100_3p
print(introns_3p.shape)
introns_3p.head()

(1252882, 5)


,chrom,start,end,transcriptID,strand
0,chr1,12512,12612,NR_046018.2_1,+
1,chr1,13120,13220,NR_046018.2_2,+
11,chr1,30875,30975,XR_001737835.1_2,+
49,chr1,183031,183131,NR_148357.1_1,+
50,chr1,183639,183739,NR_148357.1_2,+


In [117]:
# Sort introns 
introns_3p_sorted = introns_3p.sort_values(by=["chrom","start","end"])
introns_3p_sorted.head()

,chrom,start,end,transcriptID,strand
0,chr1,12512,12612,NR_046018.2_1,+
1,chr1,13120,13220,NR_046018.2_2,+
2,chr1,14829,14929,NR_024540.1_1,-
3,chr1,15038,15138,NR_024540.1_2,-
4,chr1,15947,16047,NR_024540.1_3,-


In [118]:
introns_3p_sorted_justTranscriptIDs = introns_3p_sorted.iloc[:,[3,0,1,2,4]]
introns_3p_sorted_justTranscriptIDs.columns = ["GeneID","Chr","Start","End","Strand"]
introns_3p_sorted_justTranscriptIDs.head()

,GeneID,Chr,Start,End,Strand
0,NR_046018.2_1,chr1,12512,12612,+
1,NR_046018.2_2,chr1,13120,13220,+
2,NR_024540.1_1,chr1,14829,14929,-
3,NR_024540.1_2,chr1,15038,15138,-
4,NR_024540.1_3,chr1,15947,16047,-


In [119]:
introns_3p_sorted_justTranscriptIDs["Start"] = introns_3p_sorted_justTranscriptIDs["Start"] + 1
introns_3p_sorted_justTranscriptIDs.head()

,GeneID,Chr,Start,End,Strand
0,NR_046018.2_1,chr1,12513,12612,+
1,NR_046018.2_2,chr1,13121,13220,+
2,NR_024540.1_1,chr1,14830,14929,-
3,NR_024540.1_2,chr1,15039,15138,-
4,NR_024540.1_3,chr1,15948,16047,-


In [120]:
introns_3p_sorted_justTranscriptIDs.to_csv("../data/SAFformat_ExtractedIntronCoordinatesFromGFFfile_3p_OnlyTranscriptIDs.bed",sep="\t",header=True,index=False)